<a href="https://colab.research.google.com/github/samiha-mahin/Ovarian-cancer/blob/main/HT_DenseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle tensorflow numpy pandas opencv-python matplotlib scikit-learn

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"samihamuntahamahin","key":"1ad5aaba9143fbd38da418ad8d278398"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download dataset
!kaggle datasets download -d bitsnpieces/ovarian-cancer-and-subtypes-dataset-histopathology --unzip

Dataset URL: https://www.kaggle.com/datasets/bitsnpieces/ovarian-cancer-and-subtypes-dataset-histopathology
License(s): CC-BY-SA-4.0


In [ ]:
# Import libraries
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Dataset and config
DATASET_PATH = "OvarianCancer"
classes = ["Mucinous", "Non_Cancerous", "Endometri", "Serous", "Clear_Cell"]
class_mapping = {cls: i for i, cls in enumerate(classes)}
IMG_SIZE = 224

# Load and preprocess images
X, y = [], []
for cls in classes:
    class_path = os.path.join(DATASET_PATH, cls)
    if not os.path.exists(class_path): continue
    for img_file in os.listdir(class_path):
        img_path = os.path.join(class_path, img_file)
        img = cv2.imread(img_path)
        if img is None: continue
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = tf.keras.applications.densenet.preprocess_input(img)
        X.append(img)
        y.append(class_mapping[cls])

X = np.array(X)
y = to_categorical(y, num_classes=len(classes))

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = datagen.flow(X_train, y_train, batch_size=32)

# Load DenseNet121
base_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Unfreeze deeper layers
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Custom top layers
x = GlobalAveragePooling2D()(base_model.output)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(len(classes), activation='softmax')(x)

# Model
model = Model(inputs=base_model.input, outputs=output)

# Compile
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=7, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    ModelCheckpoint("best_model_densenet121.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
]

# Train
history = model.fit(
    train_generator,
    validation_data=(X_test, y_test),
    epochs=50,
    callbacks=callbacks
)


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23s/step - accuracy: 0.2580 - loss: 2.0722 
Epoch 1: val_accuracy improved from -inf to 0.40000, saving model to best_model_densenet121.h5


13/13 ━━━━━━━━━━━━━━━━━━━━ 447s 26s/step - accuracy: 0.2630 - loss: 2.0553 - val_accuracy: 0.4000 - val_loss: 1.3955 - learning_rate: 1.0000e-04
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23s/step - accuracy: 0.4431 - loss: 1.3759 
Epoch 2: val_accuracy improved from 0.40000 to 0.59000, saving model to best_model_densenet121.h5


13/13 ━━━━━━━━━━━━━━━━━━━━ 370s 25s/step - accuracy: 0.4476 - loss: 1.3657 - val_accuracy: 0.5900 - val_loss: 1.1493 - learning_rate: 1.0000e-04
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24s/step - accuracy: 0.6190 - loss: 0.9539 
Epoch 3: val_accuracy improved from 0.59000 to 0.64000, saving model to best_model_densenet121.h5


...

13/13 ━━━━━━━━━━━━━━━━━━━━ 331s 27s/step - accuracy: 0.7350 - loss: 0.7719 - val_accuracy: 0.6900 - val_loss: 0.9164 - learning_rate: 1.0000e-04
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24s/step - accuracy: 0.8090 - loss: 0.5400 
Epoch 5: val_accuracy improved from 0.69000 to 0.71000, saving model to best_model_densenet121.h5


13/13 ━━━━━━━━━━━━━━━━━━━━ 336s 26s/step - accuracy: 0.8092 - loss: 0.5417 - val_accuracy: 0.7100 - val_loss: 0.8522 - learning_rate: 1.0000e-04
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23s/step - accuracy: 0.8247 - loss: 0.5263 
Epoch 6: val_accuracy improved from 0.71000 to 0.74000, saving model to best_model_densenet121.h5


13/13 ━━━━━━━━━━━━━━━━━━━━ 328s 25s/step - accuracy: 0.8250 - loss: 0.5249 - val_accuracy: 0.7400 - val_loss: 0.7775 - learning_rate: 1.0000e-04
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23s/step - accuracy: 0.8161 - loss: 0.4847 
Epoch 7: val_accuracy improved from 0.74000 to 0.77000, saving model to best_model_densenet121.h5


13/13 ━━━━━━━━━━━━━━━━━━━━ 321s 25s/step - accuracy: 0.8167 - loss: 0.4840 - val_accuracy: 0.7700 - val_loss: 0.6929 - learning_rate: 1.0000e-04
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23s/step - accuracy: 0.8675 - loss: 0.3493 
Epoch 8: val_accuracy improved from 0.77000 to 0.80000, saving model to best_model_densenet121.h5


13/13 ━━━━━━━━━━━━━━━━━━━━ 321s 25s/step - accuracy: 0.8672 - loss: 0.3502 - val_accuracy: 0.8000 - val_loss: 0.6157 - learning_rate: 1.0000e-04
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 23s/step - accuracy: 0.8493 - loss: 0.4175 
Epoch 9: val_accuracy did not improve from 0.80000
13/13 ━━━━━━━━━━━━━━━━━━━━ 319s 24s/step - accuracy: 0.8507 - loss: 0.4159 - val_accuracy: 0.8000 - val_loss: 0.6016 - learning_rate: 1.0000e-04
Epoch 10/50
11/13 ━━━━━━━━━━━━━━━━━━━━ 48s 24s/step - accuracy: 0.9186 - loss: 0.2806 

In [ ]:
# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ Final Test Accuracy: {accuracy*100:.2f}%")
print(f"🧪 Final Test Loss: {loss:.4f}")

In [ ]:
# Save model
model.save("densenet121_ovarian_cancer_final_model.h5")
print("✅ Model saved!")

In [ ]:
# Plot
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Train Acc")
plt.plot(history.history["val_accuracy"], label="Val Acc")
plt.legend(); plt.title("Accuracy"); plt.xlabel("Epochs"); plt.ylabel("Accuracy")

plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.legend(); plt.title("Loss"); plt.xlabel("Epochs"); plt.ylabel("Loss")
plt.show()